In [3]:
import tensorflow as tf

!pip3 install tensorflow-addons
from tensorflow_addons.text.crf import crf_log_likelihood

## if tensorflow version 1.0
# from tensorflow.contib.crf import crf_log_likelihood

In [2]:
def loss_layer(self, project_logits, lengths, name=None):
    '''
    parameters:
        - project_logits: [1, num_steps, num_tags]
        - lengths: lengths of sequence
    '''
    with tf.variable_scope("crf_loss" if not name else name):
        small = -1000.0
        # pad logits for crf loss
        start_logits = tf.concat(
            [small * tf.ones(shape=[self.batch_size, 1, self.num_tags]), tf.zeros(shape=[self.batch_size, 1, 1])], 
            axis=-1
        )
        
        pad_logits = tf.cast(
            small * tf.ones([self.batch_size, self.num_steps, 1]), 
            tf.float32)
        
        logits = tf.concat(
            [project_logits, pad_logits], 
            axis=-1)
        
        logits = tf.concat(
            [start_logits, logits], 
            axis=1)
        
        targets = tf.concat(
            [tf.cast(self.num_tags * tf.ones([self.batch_size, 1]), tf.int32), self.targets], 
            axis=-1
        )
        
        self.trans = tf.get_variable(
            "transitions",
            shape=[self.num_tags + 1, self.num_tags + 1],
            initializer=self.initializer
        )
        
        log_likelihood, self.trans = crf_log_likelihood(inputs=logits,
                                                        tag_indices=targets,
                                                        transition_params=self.tans,
                                                        sequence_lengths=lengths+1)
    return tf.reduce_mean(-log_likelihood)